In [ ]:
import fitz
from env_params import openrouter_api_key
from haystack.components.builders import PromptBuilder
import os

from openai import OpenAI
from env_params import openrouter_api_key

quiz_generation_template = """Given the following text, create {{no_questions}} multiple choice quizzes in JSON format.
Each question should have 4 different options, and only one of them should be correct.
The options should be unambiguous.
Each option should begin with a letter followed by a period and a space (e.g., "a. option").
The question should also briefly mention the general topic of the text so that it can be understood in isolation.
Each question should not give hints to answer the other questions.
Include challenging questions, which require reasoning.

respond with JSON only, no markdown or descriptions.

example JSON format you should absolutely follow:
{"topic": "a sentence explaining the topic of the text",
 "questions":
  [
    {
      "question": "text of the question",
      "options": ["a. 1st option", "b. 2nd option", "c. 3rd option", "d. 4th option"],
      "right_option": "c"  # letter of the right option ("a" for the first, "b" for the second, etc.)
    }, ...
  ]
}


text:
{% for doc in documents %}{{ doc.content|truncate(1000000) }}{% endfor %}
"""

def read_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        text += page.get_text()
    return text




In [ ]:
def get_list_of_documents(documents_paths):
    document_list = []
    for document in documents_paths:
        document_list.append({'content':read_pdf(document)})
    return document_list

builder = PromptBuilder(
    template=quiz_generation_template,
    required_variables=["documents","no_questions"]
)

document_paths = ["example_data/Module2 (1).pdf"]

In [ ]:
documents = get_list_of_documents(document_paths)
no_questions = 3
example_text = "Anatomy\nCapybaras are the largest rodents on earth. They can weigh more than 100 lbs (45 kg) and grow to be 4 feet (1.2 m) long and 2 feet (0.6 m) tall — much larger than their guinea pig relatives! They have a heavy, barrel-shaped body which sits on relatively squat legs, shorter in the front than the back. Their brown fur is coarse and sparse enough to reveal the grey skin beneath it. Capybaras have partially webbed feet, which help to propel them through the water or swampy areas. Similar to a hippopotamus, the capybara’s eyes, nose, and ears are located on the top of its head, allowing it to peek above the surface for a breath of air and a quick check for predators while the bulk of its body remains hidden beneath the water.\nWe're All In\nTogether, we're building a future where people and nature thrive. Sign up today and join our movement...\nHabitat\nCapybaras are found in Panama, Colombia, Venezuela, Guyana, Peru, Brazil, Paraguay, Northeast Argentina and Uruguay. They are semi-aquatic and will spend most of their time in dense vegetation around rivers, lakes, ponds, marshes and swamps. Feeding mostly in the afternoon, and on and off at night, capybaras tend to doze in the morning. They take short naps throughout the day while other members of their group stand guard.\nDiet\nThey are herbivores and eat the vegetation that lines water sources and other aquatic plants. An adult capybara can eat 6 to 8 lbs (2.7 to 3.6 kg) of fresh grass a day. They also eat their own poop, which contains beneficial bacteria that helps their stomach to break down the fiber from the grass. During the dry season or in drought conditions, capybaras will also eat grains, melons, reeds and squashes.\nThreats\nCapybaras are naturally threatened by jaguars, caimans and anacondas, and their young can be taken by ocelots and harpy eagles. Their main threat, however, is humans — they are hunted extensively for their meat and their hide, which can be made into leather. In some countries, the practice of farming capybaras has sprung up, relieving some of the pressure on wild populations. As with all rainforest inhabitants, deforestation also poses a threat.\nDid You Know\nWhen threatened, capybaras will jump into the water and hide beneath the surface. They can hold their breath for almost five minutes! They are also very vocal animals and communicate with each other through barks, chirps, whistles and purrs. They do this to keep track of one another and their young. They also use scent glands to mark their territory and communicate.\nSources\n- Jukofsky, Diane. Encyclopedia of Rainforests. Connecticut: Oryx Press, 2002\n- “Capybara Facts,” The Smithsonian National Zoo website, 2007\n- The San Diego Zoo\n- The IUCN Red List of Threatened Species"


builder = PromptBuilder(
    template=quiz_generation_template,
    required_variables=["documents","no_questions"]
)

prompt = builder.run(documents=documents, no_questions=no_questions)
print(prompt)




In [ ]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_api_key,
)
completion = client.chat.completions.create(
  model="deepseek/deepseek-r1-distill-llama-70b:free",
  messages=[
    {
      "role": "user",
      "content": prompt['prompt']
    }
  ]
)